Install Downloader

In [1]:
!pip -q install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.9 MB/s eta 0:00:00


Download YouTube audio as MP3

In [2]:
import os

url = "https://youtu.be/KKNCiRWd_j0?si=kucXFXw5EGpoIHw_"

!yt-dlp -x --audio-format mp3 -o "sample.%(ext)s" "$url"

print(" Files:", os.listdir())


[youtube] Extracting URL: https://youtu.be/KKNCiRWd_j0?si=kucXFXw5EGpoIHw_
[youtube] KKNCiRWd_j0: Downloading webpage
[youtube] KKNCiRWd_j0: Downloading android sdkless player API JSON
[youtube] KKNCiRWd_j0: Downloading web safari player API JSON
[youtube] KKNCiRWd_j0: Downloading m3u8 information
[info] KKNCiRWd_j0: Downloading 1 format(s): 251
[download] Destination: sample.webm
[download] 100% of   17.17MiB in 00:00:02 at 6.72MiB/s
[ExtractAudio] Destination: sample.mp3
Deleting original file sample.webm (pass -k to keep)
 Files: ['.config', 'sample.mp3', 'sample_data']


In [3]:
import os
os.listdir()


['.config', 'sample.mp3', 'sample_data']

In [4]:
!pip -q install yt-dlp

url = "https://youtu.be/KKNCiRWd_j0?si=kucXFXw5EGpoIHw_"

!yt-dlp -x --audio-format mp3 -o "sample.%(ext)s" "$url"


[youtube] Extracting URL: https://youtu.be/KKNCiRWd_j0?si=kucXFXw5EGpoIHw_
[youtube] KKNCiRWd_j0: Downloading webpage
[youtube] KKNCiRWd_j0: Downloading android sdkless player API JSON
[youtube] KKNCiRWd_j0: Downloading web safari player API JSON
[youtube] KKNCiRWd_j0: Downloading m3u8 information
[info] KKNCiRWd_j0: Downloading 1 format(s): 251
[download] sample.mp3 has already been downloaded
[ExtractAudio] Not converting audio sample.mp3; file is already in target format mp3


In [5]:
import os
os.listdir()


['.config', 'sample.mp3', 'sample_data']

In [6]:
!ls -lh


total 16M
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root  16M Jan 22 12:00 sample.mp3


Convert MP3 → WAV + preprocessing (mono + 16kHz)

In [7]:
from pydub import AudioSegment

input_audio = "sample.mp3"
audio = AudioSegment.from_file(input_audio)

audio = audio.set_channels(1)
audio = audio.set_frame_rate(16000)

audio.export("audio.wav", format="wav")
print("✅ Converted to audio.wav")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Converted to audio.wav


Chunk the audio (30 sec chunks + 2 sec overlap)

In [8]:
import os
from pydub import AudioSegment

os.makedirs("chunks", exist_ok=True)

audio = AudioSegment.from_wav("audio.wav")

chunk_length_ms = 30 * 1000
overlap_ms = 2 * 1000

chunks = []
start = 0
i = 0

while start < len(audio):
    end = start + chunk_length_ms
    chunk = audio[start:end]
    chunk_name = f"chunks/chunk_{i:03d}.wav"
    chunk.export(chunk_name, format="wav")
    chunks.append(chunk_name)
    start = end - overlap_ms
    i += 1

print(f"Total chunks created: {len(chunks)}")
print("First 5 chunks:", chunks[:5])


Total chunks created: 48
First 5 chunks: ['chunks/chunk_000.wav', 'chunks/chunk_001.wav', 'chunks/chunk_002.wav', 'chunks/chunk_003.wav', 'chunks/chunk_004.wav']


In [9]:
!pip -q install faster-whisper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


In [10]:
import os, json
from faster_whisper import WhisperModel

# Load model (small is fast + good accuracy)
model = WhisperModel("small", device="cuda", compute_type="float16")

chunk_files = sorted([f"chunks/{f}" for f in os.listdir("chunks") if f.endswith(".wav")])

transcripts = []

for idx, file in enumerate(chunk_files):
    print(f" Transcribing: {file}")

    segments, info = model.transcribe(file, language="en")

    chunk_text = ""
    for seg in segments:
        chunk_text += seg.text.strip() + " "

    transcripts.append({
        "chunk_id": idx,
        "file": file,
        "text": chunk_text.strip()
    })

print("✅ Transcription done. Total chunks:", len(transcripts))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

 Transcribing: chunks/chunk_000.wav
 Transcribing: chunks/chunk_001.wav
 Transcribing: chunks/chunk_002.wav
 Transcribing: chunks/chunk_003.wav
 Transcribing: chunks/chunk_004.wav
 Transcribing: chunks/chunk_005.wav
 Transcribing: chunks/chunk_006.wav
 Transcribing: chunks/chunk_007.wav
 Transcribing: chunks/chunk_008.wav
 Transcribing: chunks/chunk_009.wav
 Transcribing: chunks/chunk_010.wav
 Transcribing: chunks/chunk_011.wav
 Transcribing: chunks/chunk_012.wav
 Transcribing: chunks/chunk_013.wav
 Transcribing: chunks/chunk_014.wav
 Transcribing: chunks/chunk_015.wav
 Transcribing: chunks/chunk_016.wav
 Transcribing: chunks/chunk_017.wav
 Transcribing: chunks/chunk_018.wav
 Transcribing: chunks/chunk_019.wav
 Transcribing: chunks/chunk_020.wav
 Transcribing: chunks/chunk_021.wav
 Transcribing: chunks/chunk_022.wav
 Transcribing: chunks/chunk_023.wav
 Transcribing: chunks/chunk_024.wav
 Transcribing: chunks/chunk_025.wav
 Transcribing: chunks/chunk_026.wav
 Transcribing: chunks/chunk_

In [11]:
import json

# Save chunk-wise transcript
with open("chunk_transcripts.json", "w", encoding="utf-8") as f:
    json.dump(all_transcripts, f, indent=2, ensure_ascii=False)

print("✅ Saved: chunk_transcripts.json")


NameError: name 'all_transcripts' is not defined

In [12]:
globals().keys()


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_exit_code', '_i2', 'os', 'url', '_i3', '_3', '_i4', '_i5', '_5', '_i6', '_i7', 'AudioSegment', 'input_audio', 'audio', '_i8', 'chunk_length_ms', 'overlap_ms', 'chunks', 'start', 'i', 'end', 'chunk', 'chunk_name', '_i9', '_i10', 'json', 'WhisperModel', 'model', 'chunk_files', 'transcripts', 'idx', 'file', 'segments', 'info', 'chunk_text', 'seg', '_i11', 'f', '_i12'])

In [13]:
import os
from faster_whisper import WhisperModel

# ✅ Use CPU safe settings (works always)
model = WhisperModel("small", device="cpu", compute_type="int8")

chunk_files = sorted([f"chunks/{f}" for f in os.listdir("chunks") if f.endswith(".wav")])

all_transcripts = []

for idx, chunk_path in enumerate(chunk_files):
    print(f"🎧 Transcribing: {chunk_path}")

    segments, info = model.transcribe(chunk_path, beam_size=5)

    chunk_text = " ".join([seg.text.strip() for seg in segments]).strip()

    all_transcripts.append({
        "chunk_id": idx,
        "chunk_file": chunk_path,
        "text": chunk_text
    })

print("✅ Transcription done. Total chunks:", len(all_transcripts))


🎧 Transcribing: chunks/chunk_000.wav
🎧 Transcribing: chunks/chunk_001.wav
🎧 Transcribing: chunks/chunk_002.wav
🎧 Transcribing: chunks/chunk_003.wav
🎧 Transcribing: chunks/chunk_004.wav
🎧 Transcribing: chunks/chunk_005.wav
🎧 Transcribing: chunks/chunk_006.wav
🎧 Transcribing: chunks/chunk_007.wav
🎧 Transcribing: chunks/chunk_008.wav
🎧 Transcribing: chunks/chunk_009.wav
🎧 Transcribing: chunks/chunk_010.wav
🎧 Transcribing: chunks/chunk_011.wav
🎧 Transcribing: chunks/chunk_012.wav
🎧 Transcribing: chunks/chunk_013.wav
🎧 Transcribing: chunks/chunk_014.wav
🎧 Transcribing: chunks/chunk_015.wav
🎧 Transcribing: chunks/chunk_016.wav
🎧 Transcribing: chunks/chunk_017.wav
🎧 Transcribing: chunks/chunk_018.wav
🎧 Transcribing: chunks/chunk_019.wav
🎧 Transcribing: chunks/chunk_020.wav
🎧 Transcribing: chunks/chunk_021.wav
🎧 Transcribing: chunks/chunk_022.wav
🎧 Transcribing: chunks/chunk_023.wav
🎧 Transcribing: chunks/chunk_024.wav
🎧 Transcribing: chunks/chunk_025.wav
🎧 Transcribing: chunks/chunk_026.wav
🎧

In [14]:
import json

with open("chunk_transcripts.json", "w", encoding="utf-8") as f:
    json.dump(all_transcripts, f, indent=2, ensure_ascii=False)

print("✅ Saved: chunk_transcripts.json")


✅ Saved: chunk_transcripts.json


In [15]:
full_text = "\n".join([x["text"].strip() for x in all_transcripts if x["text"].strip()])

with open("full_transcript.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print("✅ Saved: full_transcript.txt")
print("\n--- Transcript Preview ---\n")
print(full_text[:1200])


✅ Saved: full_transcript.txt

--- Transcript Preview ---

I want to tell you what I see coming. I've been lucky enough to be working on AI for almost 15 years now. Back when I started to describe it as fringe would be an understatement. Researchers would say, no, no, we're only working on machine learning because working on AI was seen as way too out there. In 2010, just a very mention of the phrase AGI, artificial.
to raise AGI, artificial general intelligence, would get you some seriously strange looks. And even a cold shoulder. You're actually building AGI, people would say. Isn't that something out of science fiction? People thought it was 50 years away or 100 years away, if it was even possible at all. Talk of AI was, I guess, kind of embarrassing. People generally thought we were weird. And I guess in some ways we kind of were. It wasn't long though.
It wasn't long, though, before AI started beating humans at a whole range of tasks that people previously thought were way out of r

Clean Transcript

In [16]:
import re

def clean_text(text: str) -> str:
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)  # remove extra spaces
    text = re.sub(r"\b(um+|uh+|hmm+|erm+)\b", "", text, flags=re.I)  # fillers
    return text.strip()

cleaned_text = clean_text(full_text)

with open("cleaned_transcript.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print("✅ Saved: cleaned_transcript.txt")
print("\n--- Cleaned Transcript Preview ---\n")
print(cleaned_text[:1000])


✅ Saved: cleaned_transcript.txt

--- Cleaned Transcript Preview ---

I want to tell you what I see coming. I've been lucky enough to be working on AI for almost 15 years now. Back when I started to describe it as fringe would be an understatement. Researchers would say, no, no, we're only working on machine learning because working on AI was seen as way too out there. In 2010, just a very mention of the phrase AGI, artificial. to raise AGI, artificial general intelligence, would get you some seriously strange looks. And even a cold shoulder. You're actually building AGI, people would say. Isn't that something out of science fiction? People thought it was 50 years away or 100 years away, if it was even possible at all. Talk of AI was, I guess, kind of embarrassing. People generally thought we were weird. And I guess in some ways we kind of were. It wasn't long though. It wasn't long, though, before AI started beating humans at a whole range of tasks that people previously thought were w

In [17]:
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(cleaned_text)

print("✅ Total sentences:", len(sentences))
print("\n--- First 10 sentences ---\n")
for s in sentences[:10]:
    print("-", s)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Sentence Splitting

In [18]:
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Generate Sentence Embeddings

In [19]:
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(cleaned_text)

print("✅ Total sentences:", len(sentences))
print("\n--- First 10 sentences ---\n")
for s in sentences[:10]:
    print("-", s)


✅ Total sentences: 273

--- First 10 sentences ---

- I want to tell you what I see coming.
- I've been lucky enough to be working on AI for almost 15 years now.
- Back when I started to describe it as fringe would be an understatement.
- Researchers would say, no, no, we're only working on machine learning because working on AI was seen as way too out there.
- In 2010, just a very mention of the phrase AGI, artificial.
- to raise AGI, artificial general intelligence, would get you some seriously strange looks.
- And even a cold shoulder.
- You're actually building AGI, people would say.
- Isn't that something out of science fiction?
- People thought it was 50 years away or 100 years away, if it was even possible at all.


Sentence Embeddings

In [20]:
import re

sentences = re.split(r'(?<=[.!?])\s+', cleaned_text)
sentences = [s.strip() for s in sentences if len(s.strip()) > 0]

print("✅ Total sentences:", len(sentences))
print(sentences[:10])


✅ Total sentences: 279
['I want to tell you what I see coming.', "I've been lucky enough to be working on AI for almost 15 years now.", 'Back when I started to describe it as fringe would be an understatement.', "Researchers would say, no, no, we're only working on machine learning because working on AI was seen as way too out there.", 'In 2010, just a very mention of the phrase AGI, artificial.', 'to raise AGI, artificial general intelligence, would get you some seriously strange looks.', 'And even a cold shoulder.', "You're actually building AGI, people would say.", "Isn't that something out of science fiction?", 'People thought it was 50 years away or 100 years away, if it was even possible at all.']


In [21]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embed_model.encode(sentences, show_progress_bar=True)

print("✅ Embeddings created successfully!")
print("✅ Shape:", embeddings.shape)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

✅ Embeddings created successfully!
✅ Shape: (279, 384)


Topic Segmentation (Cosine Similarity Drop)

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# similarity between consecutive sentences
sims = []
for i in range(len(embeddings)-1):
    sim = cosine_similarity([embeddings[i]], [embeddings[i+1]])[0][0]
    sims.append(sim)

sims = np.array(sims)

# dynamic threshold
threshold = np.percentile(sims, 25)   # 25th percentile

boundaries = [0]
for i, sim in enumerate(sims):
    if sim < threshold:
        boundaries.append(i+1)

boundaries.append(len(sentences))
boundaries = sorted(list(set(boundaries)))

print("✅ Similarity threshold:", round(float(threshold), 4))
print("✅ Total topics detected:", len(boundaries)-1)
print("✅ Boundaries (first few):", boundaries[:15])


✅ Similarity threshold: 0.1509
✅ Total topics detected: 71
✅ Boundaries (first few): [0, 1, 2, 3, 6, 7, 8, 10, 16, 18, 21, 22, 23, 26, 27]


Create Topic Blocks

In [23]:
topics = []
for i in range(len(boundaries)-1):
    start = boundaries[i]
    end = boundaries[i+1]
    topic_text = " ".join(sentences[start:end]).strip()

    topics.append({
        "topic_id": i+1,
        "start_sentence": start,
        "end_sentence": end-1,
        "text": topic_text
    })

print("✅ Topics created:", len(topics))

print("\n--- Topic 1 Preview ---\n")
print(topics[0]["text"][:1200])


✅ Topics created: 71

--- Topic 1 Preview ---

I want to tell you what I see coming.


In [24]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: similarities
sims = []
for i in range(len(embeddings)-1):
    sim = cosine_similarity([embeddings[i]], [embeddings[i+1]])[0][0]
    sims.append(sim)

sims = np.array(sims)

# Step 2: stricter threshold (makes fewer boundaries)
threshold = np.percentile(sims, 10)  # try 10% (less sensitive)

# Step 3: minimum topic size (very important)
min_sentences_per_topic = 8

boundaries = [0]
last_boundary = 0

for i, sim in enumerate(sims):
    # boundary only if similarity is low AND enough sentences passed
    if sim < threshold and (i + 1 - last_boundary) >= min_sentences_per_topic:
        boundaries.append(i+1)
        last_boundary = i+1

boundaries.append(len(sentences))
boundaries = sorted(list(set(boundaries)))

print("✅ Similarity threshold:", round(float(threshold), 4))
print("✅ Total topics detected:", len(boundaries)-1)
print("✅ Boundaries:", boundaries[:15])


✅ Similarity threshold: 0.0826
✅ Total topics detected: 15
✅ Boundaries: [0, 21, 29, 46, 69, 81, 89, 107, 117, 133, 161, 170, 199, 233, 249]


In [25]:
topics = []
for i in range(len(boundaries)-1):
    start = boundaries[i]
    end = boundaries[i+1]
    topic_text = " ".join(sentences[start:end]).strip()

    topics.append({
        "topic_id": i+1,
        "start_sentence": start,
        "end_sentence": end-1,
        "num_sentences": end-start,
        "text": topic_text
    })

print("✅ Topics created:", len(topics))
print("\n--- Topic 1 Preview ---\n")
print(topics[0]["text"][:1200])


✅ Topics created: 15

--- Topic 1 Preview ---

I want to tell you what I see coming. I've been lucky enough to be working on AI for almost 15 years now. Back when I started to describe it as fringe would be an understatement. Researchers would say, no, no, we're only working on machine learning because working on AI was seen as way too out there. In 2010, just a very mention of the phrase AGI, artificial. to raise AGI, artificial general intelligence, would get you some seriously strange looks. And even a cold shoulder. You're actually building AGI, people would say. Isn't that something out of science fiction? People thought it was 50 years away or 100 years away, if it was even possible at all. Talk of AI was, I guess, kind of embarrassing. People generally thought we were weird. And I guess in some ways we kind of were. It wasn't long though. It wasn't long, though, before AI started beating humans at a whole range of tasks that people previously thought were way out of reach, under

Topic Labeling using TF-IDF Keywords

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

topic_texts = [t["text"] for t in topics]

vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, ngram_range=(1,2))
tfidf = vectorizer.fit_transform(topic_texts)
feature_names = np.array(vectorizer.get_feature_names_out())

def get_topic_label(tfidf_row, top_n=4):
    scores = tfidf_row.toarray().flatten()
    top_idx = scores.argsort()[-top_n:][::-1]
    keywords = feature_names[top_idx]
    return " / ".join(keywords)

for i, t in enumerate(topics):
    t["topic_label"] = get_topic_label(tfidf[i], top_n=4)

print("✅ Topic Labels Generated!\n")
for t in topics:
    print(f"Topic {t['topic_id']}: {t['topic_label']}")


✅ Topic Labels Generated!

Topic 1: working / agi / people / ai
Topic 2: actually simpler / simpler / simpler kind / kind
Topic 3: like / gets / ll like / ai
Topic 4: don / try / let / year
Topic 5: began / went / life / went produce
Topic 6: speeding / wave / fastest / fastest consequential
Topic 7: words / ago / years ago / parameters
Topic 8: ll / gave / cloud based / cloud
Topic 9: pocket / ability / ability actually / business
Topic 10: saying / tools / think / start
Topic 11: inspire / ai want / want build / build
Topic 12: new / ai / oil / opened
Topic 13: isn / ai isn / amazing / ai
Topic 14: obviously / risk / think / operates obviously
Topic 15: design / right / thank / think


Summarization per Topic

In [27]:
!pip -q install transformers


In [28]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(text, max_len=120, min_len=40):
    # summarizer works better on limited length text
    text = text.strip()
    if len(text.split()) < 60:
        return text

    text = text[:3500]  # limit for model
    summary = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
    return summary[0]["summary_text"]

# topic-wise summaries
for t in topics:
    t["summary"] = summarize_text(t["text"], max_len=120, min_len=40)

print("✅ Topic Summaries Created!\n")
print("Example summary:\n")
print("Label:", topics[0]["topic_label"])
print("Summary:", topics[0]["summary"])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Your max_length is set to 120, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Topic Summaries Created!

Example summary:

Label: working / agi / people / ai
Summary: In 2010, just a very mention of the phrase AGI, artificial general intelligence, would get you some seriously strange looks. People thought it was 50 years away or 100 years away, if it was even possible at all. It wasn't long before AI started beating humans at a whole range of tasks.


Save Final Output JSON

In [29]:
import json

final_output = {
    "total_topics": len(topics),
    "topics": topics
}

with open("final_topics_output.json", "w", encoding="utf-8") as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print("✅ Saved: final_topics_output.json")


✅ Saved: final_topics_output.json


In [30]:
!pip -q install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 144.4 MB/s eta 0:00:00


In [36]:
%%writefile app.py
import os
import json
import streamlit as st

st.set_page_config(page_title="Audio Analysis Dashboard", layout="wide")

st.title("🎙️ Automated Podcast Transcription & Topic Segmentation")
st.subheader("Audio Analysis")

# --------- Helper functions ----------
def load_text_file(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    return ""

def load_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    return None

# --------- Sidebar ----------
st.sidebar.header(" Navigation")
section = st.sidebar.radio("Go to", ["Overview", "Transcript", "Topics & Summaries", "Download Outputs"])

# --------- Files present ----------
transcript_path = "full_transcript.txt"
cleaned_path = "cleaned_transcript.txt"
topics_json_path = "final_topics_output.json"

full_transcript = load_text_file(transcript_path)
cleaned_transcript = load_text_file(cleaned_path)
topics_data = load_json(topics_json_path)

# --------- Overview ----------
if section == "Overview":
    st.info("✅ This dashboard shows transcript, topics, labels, and summaries generated from podcast/TED audio.")

    col1, col2, col3 = st.columns(3)
    col1.metric("Transcript Available", "Yes" if len(full_transcript) > 0 else "No")
    col2.metric("Cleaned Transcript Available", "Yes" if len(cleaned_transcript) > 0 else "No")
    col3.metric("Topics JSON Available", "Yes" if topics_data else "No")

    st.markdown("### 📂 Current Project Files")
    st.write(os.listdir("."))

# --------- Transcript ----------
elif section == "Transcript":
    st.markdown("## 📝 Transcript Viewer")
    tab1, tab2 = st.tabs(["Full Transcript", "Cleaned Transcript"])

    with tab1:
        if len(full_transcript) == 0:
            st.warning("full_transcript.txt not found.")
        else:
            st.text_area("Full Transcript", full_transcript, height=450)

    with tab2:
        if len(cleaned_transcript) == 0:
            st.warning("cleaned_transcript.txt not found.")
        else:
            st.text_area("Cleaned Transcript", cleaned_transcript, height=450)

# --------- Topics ----------
elif section == "Topics & Summaries":
    st.markdown("## 🧩 Topics & Summaries")

    if not topics_data:
        st.error("final_topics_output.json not found. Please generate topics first.")
    else:
        total_topics = topics_data.get("total_topics", 0)
        topics = topics_data.get("topics", [])

        st.success(f"✅ Total Topics: {total_topics}")

        # Search filter
        query = st.text_input("🔎 Search in topic labels / summaries", "")

        for t in topics:
            label = t.get("topic_label", f"Topic {t.get('topic_id')}")
            summary = t.get("summary", "")
            text = t.get("text", "")

            searchable = (label + " " + summary + " " + text).lower()
            if query.strip() and query.lower() not in searchable:
                continue

            with st.expander(f"📌 Topic {t.get('topic_id')} — {label}", expanded=False):
                c1, c2 = st.columns([1, 1])

                with c1:
                    st.markdown("### ✅ Summary")
                    st.write(summary if summary else "(No summary found)")

                with c2:
                    st.markdown("### 📄 Topic Text")
                    st.write(text[:2500] + ("..." if len(text) > 2500 else ""))

# --------- Download ----------
elif section == "Download Outputs":
    st.markdown("## ⬇️ Download Outputs")

    if os.path.exists(topics_json_path):
        with open(topics_json_path, "rb") as f:
            st.download_button("Download Topics JSON", f, file_name="final_topics_output.json")

    if os.path.exists(transcript_path):
        with open(transcript_path, "rb") as f:
            st.download_button("Download Full Transcript", f, file_name="full_transcript.txt")

    if os.path.exists(cleaned_path):
        with open(cleaned_path, "rb") as f:
            st.download_button("Download Cleaned Transcript", f, file_name="cleaned_transcript.txt")

    st.caption("")


Overwriting app.py


In [32]:
from pyngrok import ngrok
import subprocess
import time

# Start streamlit in background
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])
time.sleep(3)

# Create public url
public_url = ngrok.connect(8501)
print("✅ Streamlit Dashboard URL:", public_url)


ERROR:pyngrok.process.ngrok:t=2026-01-22T13:04:58+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2026-01-22T13:04:58+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2026-01-22T13:04:58+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [33]:
!ngrok config add-authtoken 38O80FeMvlOevnkLpBMyZh7BqHy_27nmk52Ck6nimfZ7YKX52


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
!pkill -f streamlit


In [37]:
from pyngrok import ngrok
import subprocess
import time

process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])
time.sleep(3)

public_url = ngrok.connect(8501)
print("✅ Streamlit Dashboard URL:", public_url)


✅ Streamlit Dashboard URL: NgrokTunnel: "https://edgy-grayce-evidently.ngrok-free.dev" -> "http://localhost:8501"


In [38]:
import os
os.listdir()


['.config',
 'audio.wav',
 'cleaned_transcript.txt',
 'app.py',
 'chunks',
 'sample.mp3',
 'final_topics_output.json',
 'full_transcript.txt',
 'chunk_transcripts.json',
 'sample_data']

In [39]:
%%writefile requirements.txt
streamlit
pyngrok
pydub
librosa
soundfile
ffmpeg-python
faster-whisper
nltk
sentence-transformers
scikit-learn
transformers
yt-dlp


Writing requirements.txt


In [40]:
import os
os.listdir()


['.config',
 'audio.wav',
 'cleaned_transcript.txt',
 'app.py',
 'chunks',
 'requirements.txt',
 'sample.mp3',
 'final_topics_output.json',
 'full_transcript.txt',
 'chunk_transcripts.json',
 'sample_data']

In [41]:
from google.colab import files

files.download("app.py")
files.download("requirements.txt")
files.download("final_topics_output.json")
files.download("chunk_transcripts.json")
files.download("full_transcript.txt")
files.download("cleaned_transcript.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>